In [1]:
print("Jupyter is ready for CE40248")

Jupyter is ready for CE40248


Input -> Calculate forces and integrate equations of motion -> perform P, control T if necessary -> t = t + deltT -> Calculate and update the molecule position (-> t<tmax back to the calcualte forces tab) -> Output: key output is a trajectory file with positions at a given time, here the thermodynamics and transport properties are determined....

Sigma is the diameter of atom and epsilon is the energy of the particle

In order to control reaction in terms of energy, need to completely isolate the system. This is not physically 100% possible. Come with a theoretical limit. 

If try to determine a parameter, do not use the equilibration zone

Limitations - not considering rotational or vibrational energy

CE40248 Coursework: Molecular Dynamics
Author: Alex Pooley
Date edited: 27/10/2025
Description: Code takes a series of parameters within a system and using the Lennard-Jones potential, calculates force and velocity to create a file of positions in space, for each molecule for each time step.

In [2]:
# Import packages needed
import numpy as np
import matplotlib.pyplot as plt

import numba
from numba import jit, prange # prange is the parallel version of range
from numba.typed import List
from numba.core import atomic

ImportError: cannot import name 'atomic' from 'numba.core' (C:\Users\alexj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numba\core\__init__.py)

In [ ]:
# Lennard Jones function

@jit(nopython=True)
def U_LJ(r, sigma, epsilon): 
    # Function to calculate potential energy, using LJ potential.
    U = 4 * epsilon * ((sigma/r)**12 - (sigma/r)**6) # Dimensional U equation
    return U

@jit(nopython=True)
def F_LJ(r, sigma, epsilon): 
    # Function to calculate force magnitude, using LJ potential.
    F = 24 * epsilon *(2*(sigma/r)**12 - (sigma/r)**6) / r # Dimensional F equation
    return F/r # Returns the F over r you can multiply by the displacement vector later on

In [ ]:
# Periodic Boundary Conditions
@jit(nopython=True)
def pbc_wrap(pos, L):
    # Function to wrap around to the box if the particle flies out, mimics bulk flow now
    new_pos = pos -L * np.floor(pos / L) # Called if out of the box and position is flipped to oppostie side
    return new_pos

@jit(nopython=True)
def MIC(vec, L):
    # Function to ensure behaviour of bulk flow matched to simulation
    mic = vec - L * np.round(vec / L) # Means always measures the nearest periodic boundary, other than long way around
    return mic

In [ ]:
# Vertlet neighbour and cell neightbour listing 

def build_neighbour_list(positions, L, rc_list):
    
    N = positions.shape[0]
    nlist = [[] for _ in range (N)] # Intialise an empty neighbour list
    
    for i in range (N-1): # Compute pairwise displacement vectors
        rij = positions[i+1:] - positions[i]  # Find distance between each mol
        rij = MIC(rij, L)                     # Apply mic...
        dist2 = np.sum(rij*rij, axis=1)       # Calculate the squared distances
        mask = dist2 < rc_list**2             # If distance below cut off, take into account
        js = np.nonzero(mask)[0] + (i + 1)
        
        for j in js:
            nlist[i].append(j) # Appends each neighbours list if theyre a neighbour
    
    return nlist

@jit(nopython=True)
def get_cell_grid_params(L, rc_list):
    # Determines how many cells we need to make 
    
    # Find number of cells in one dimension
    n_cells_per_dim = int(np.floor(L / rc_list)) # How many r cutt offs fit in the L
    n_cells_per_dim = max(n_cells_per_dim, 1)    # If L is very small, we might get 0, so ensure at least 1 cell

    # Total number of cells
    total_cells = n_cells_per_dim**3 # As lloking at a cuboid
    
    return n_cells_per_dim, total_cells

@jit(nopython=True, parallel=True)
def build_neighbour_list_cells(positions, L, rc_list):
    """
    Builds the neighbour list using an O(N) cell list algorithm.
    (Numba-compiled, PARALLEL version)
    """
    N = positions.shape[0]
    nlist = List([List.empty_list(numba.int64) for _ in range(N)])
    
    n_cells_per_dim, total_cells = get_cell_grid_params(L, rc_list)
    
    head = np.full(total_cells, -1, dtype=np.int64)
    linked_list = np.full(N, -1, dtype=np.int64)

    # --- Binning Step (O(N)) ---
    # This loop CANNOT be parallel, it has a race condition
    for i in range(N):
        pos_i = positions[i]
        
        # ... (same code as before for ic, jc, kc) ...
        max_idx_float = float(n_cells_per_dim - 1)
        ic_val = (pos_i[0] / L) * n_cells_per_dim
        ic = int(max(0.0, min(ic_val, max_idx_float)))
        jc_val = (pos_i[1] / L) * n_cells_per_dim
        jc = int(max(0.0, min(jc_val, max_idx_float)))
        kc_val = (pos_i[2] / L) * n_cells_per_dim
        kc = int(max(0.0, min(kc_val, max_idx_float)))
        
        cell_idx = (kc * n_cells_per_dim + jc) * n_cells_per_dim + ic
        
        linked_list[i] = head[cell_idx]
        head[cell_idx] = i

    # --- Search Step (O(N)) ---
    # This loop IS parallel-safe. Change range to prange
    for i in prange(N):
        pos_i = positions[i]

        # ... (same code as before for finding home ic, jc, kc) ...
        max_idx_float = float(n_cells_per_dim - 1)
        ic_val = (pos_i[0] / L) * n_cells_per_dim
        ic = int(max(0.0, min(ic_val, max_idx_float)))
        jc_val = (pos_i[1] / L) * n_cells_per_dim
        jc = int(max(0.0, min(jc_val, max_idx_float)))
        kc_val = (pos_i[2] / L) * n_cells_per_dim
        kc = int(max(0.0, min(kc_val, max_idx_float)))

        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                for dz in [-1, 0, 1]:
                    # ... (rest of the loop is identical) ...
                    neighbor_ic = (ic + dx) % n_cells_per_dim
                    neighbor_jc = (jc + dy) % n_cells_per_dim
                    neighbor_kc = (kc + dz) % n_cells_per_dim
                    
                    neighbor_cell_idx = (neighbor_kc * n_cells_per_dim + neighbor_jc) * n_cells_per_dim + neighbor_ic
                    
                    j = head[neighbor_cell_idx]
                    
                    while j != -1:
                        if i < j:
                            rij_vec = pos_i - positions[j]
                            rij_vec = MIC(rij_vec, L)
                            dist2 = rij_vec[0]**2 + rij_vec[1]**2 + rij_vec[2]**2
                            
                            if dist2 < rc_list**2:
                                nlist[i].append(j)
                        
                        j = linked_list[j]
    return nlist



In [ ]:
# Add parallel=True to the decorator
@jit(nopython=True, parallel=True)
def compute_forces(positions, rc, sigma, epsilon, nlist, L):
    N = positions.shape[0]
    forces = np.zeros_like(positions)
    
    # --- POTENTIAL FIX ---
    # Create a 1-element array for potential
    # so we can atomically add to it
    potential_array = np.zeros(1) 
    # ---------------------

    # Use prange for the parallel loop
    for i in prange(N - 1):
        for j in nlist[i]:
            rij = positions[i] - positions[j]
            rij = MIC(rij, L)
            
            r2 = rij[0]**2 + rij[1]**2 + rij[2]**2
            r = np.sqrt(r2)

            if r < 1e-12:
                continue

            if r < rc:
                fij_mag = F_LJ(r, sigma, epsilon)
                fij_vec_x = fij_mag * rij[0]
                fij_vec_y = fij_mag * rij[1]
                fij_vec_z = fij_mag * rij[2]
                
                # --- SAFE WRITE to forces[i] ---
                # This is safe, no atomic needed
                forces[i, 0] += fij_vec_x
                forces[i, 1] += fij_vec_y
                forces[i, 2] += fij_vec_z
                
                # --- UNSAFE WRITE to forces[j] ---
                # Must use numba.atomic.add
                atomic.add(forces, (j, 0), -fij_vec_x)
                atomic.add(forces, (j, 1), -fij_vec_y)
                atomic.add(forces, (j, 2), -fij_vec_z)
                # ----------------------------------
                
                # --- UNSAFE WRITE to potential ---
                # Must use numba.atomic.add
                atomic.add(potential_array, 0, U_LJ(r, sigma, epsilon))
                # ----------------------------------
    
    # Return the final forces and the value from the potential array
    return forces, potential_array[0]


In [ ]:
# Max displacement function
@jit(nopython=True)
def max_displacement(p, p_ref, L):
    
    disp = MIC(p - p_ref, L)
    
    return np.max(np.linalg.norm(disp, axis=1))

In [ ]:
# Radial distribution function

def radial_distribution(traj, L, dr=0.02, rmax=None):
    pos = traj[-1]
    N = pos.shape[0]
    if rmax is None:
        rmax = 0.5*L                   # Sets the maximum radius to calculate over 
        nbins = int(np.floor(rmax/dr)) # How many thin spherical shells we need 
    
    # pair distances with MIC
    dists = []
    for i in range(N-1): # Loops through every particle
        rij = pos[i+1:] - pos[i]
        rij = MIC(rij, L) # Minimum image convention... 
        r = np.linalg.norm(rij, axis=1)
        dists.append(r)
    dists = np.concatenate(dists, axis=0)
    
    # keep only distances in (0, rmax)
    dists = dists[(dists > 0.0) & (dists < rmax)]

    # histogram
    hist, edges = np.histogram(dists, bins=nbins, range=(0.0, rmax))
    r_centers = 0.5*(edges[1:] + edges[:-1])
    
    # normalization
    rho = N / (L**3)  # Bulk number density
    shell_vol = 4.0*np.pi * (r_centers**2) * dr # Calculates each volume of the shells
    ideal = rho * N * shell_vol # Calculates expected if ideal gas
    g = (2.0 * hist) / ideal # factor 2 for i-j and j-i, multipied by two to get the actual pair count as only worked out one earlier
    
    return r_centers, g

In [ ]:
def write_extended_xyz(filename, frames, L):
    # Extended XYZ: line 2 can include "Lattice" and "Properties"
    with open(filename, "w") as f:
        for k, frame in enumerate(frames):
            N = frame.shape[0]
            f.write(f"{N}\n")
            # cubic cell matrix
            f.write(f"Lattice=\"{L} 0 0 0 {L} 0 0 0 {L}\" Properties=species:S:1:pos:R:3 Step={k}\n")
            for x, y, z in frame:
                f.write(f"Ar {x:.6f} {y:.6f} {z:.6f}\n")

Script File...

In [ ]:
# Initial Variables:
# =================================================
# MC Variables
N = 10                    # Number of particles
nsteps = 20000            # Number of integration steps, where dt=0.005tau and tau = sigma*sqrt(mass/epsilon)
dimensionless_dt = 0.001  # Dimensionless time step

# Material Variables - To be chnaged when a real substance involved
sigma = 1 #3.405 * 10**-10   # Distance at which U is zero [REF]
epsilon = 1 # 1.65 * 10**-21  # Depth of potential well [REF]
mass_atom = 1 # 6.63e-26      # Mass of one Aargon atom [kg] [REF]

# Initial Calculations
L = 5.0 * sigma                            # Box length
tau = sigma * np.sqrt(mass_atom / epsilon) # Tau calculation
dt = dimensionless_dt * tau                # Dimensional time step [s]
rc = 2.5 * sigma # Reduced cutoff (no tail corrections), take as theres no interactions when th distance between the particles is this long
kB = 1 # 1.380649e-23 # Boltz coefficient

# PIC variables 
rho = 0.80   # reduced density
Ngrid = 6    # Particles on the simple cubic lattice 
N = Ngrid**3 # As cubic (216 for now)
L = (N / rho)**(1/3) # So the cube gives the desired density 
a = L / Ngrid
coords = np.linspace(0.5 * a, L - 0.5*a, Ngrid)
x, y, z = np.meshgrid(coords, coords, coords, indexing='ij')
positions = np.vstack([x.ravel(), y.ravel(), z.ravel()]).T
np.random.seed(2)
positions += 0.05*a * (np.random.rand(N,3) - 0.5)
positions = pbc_wrap(positions, L)

# Initial velocities
velocities = np.random.randn(N, 3)           # Calculates intial velocity
velocities -= velocities.mean(axis=0)        # Removes energy drift and sets momentum to zero

rskin = 0.3             # Gives the buffer zone
rlist = rskin + rc      # Means particles not missed out in the timestep within the neighbour cell 

KE, PE, TE = [], [], [] # Lists to store energies
save_stride = 10        # Only saves every 10th frame to reduce file size
traj = []               # list of snapshots for XYZ export
steps = 500             # Number of steps for the integrator

In [ ]:
# Vertlet neighbour listing 
neighbour_list = build_neighbour_list_cells(positions, L, rlist) # Note this is a cell list!
ref_positions = positions.copy() # reference for skin criterion

In [ ]:
# Call the initial compute forces function

print("Number of particles:", N)
print("Box length L:", L)
print("Cutoff rc:", rc)
print("Neighbour list lengths (first 10):", [len(n) for n in neighbour_list[:10]])

forces, pot = compute_forces(positions, rc, sigma, epsilon, neighbour_list, L)

print("Initial potential energy =", pot)
print("Initial total force magnitude =", np.sum(np.linalg.norm(forces, axis=1)))

In [ ]:
# Velocity Vertlet Integrator - Runge Kutta?

for step in range(steps): # Will run for number of step initialised earlier
    # Velocity-Verlet algorithm 
    velocities += 0.5 * (forces/mass_atom) * dt                 # Originally the random value, F = ma rearrangement
    positions += velocities * dt                                # Then update positions accordingly due to timestep
    
    # Rebuild the neighbour list
    if max_displacement(positions, ref_positions, L) > 0.5*rskin:
        neighbour_list = build_neighbour_list_cells(positions, L, rlist)
        ref_positions = positions.copy()
    
    # Compute new forces
    forces, pot = compute_forces(positions, rc, sigma, epsilon, neighbour_list, L) # Call the forces function, now with new positions
    velocities += 0.5 * (forces/mass_atom) * dt                                    # As above...
   
    kin = 0.5 * mass_atom * np.sum(velocities**2)        # From basic kinetic equation, take into account all particles
    KE.append(kin); PE.append(pot); TE.append(kin + pot) # Add on the newly calculates parameters

    # Velocity adjustment thermostat
    Tinst_history = []
    Tinst = (2 * kin) / (3 * N * kB) 
    print(Tinst)                                         # Calculate the instantaneous temperature 
    Tinst_history.append(Tinst)                          # Add the inst temps to list for analysis
    
    # Rescale velocities to target temperature
    T = 1                              # Target temperature for system to reach
    if step % 5 == 0:
        thermostat_scaler = np.sqrt(T / Tinst) # Make a scaling coefficient for velocity
        velocities *= thermostat_scaler        # Scale the velocity to conserve temperature
    
    if step % save_stride == 0:       # If step is a multiple of the save_stride (10)
        traj.append(positions.copy()) # Only saves if mulitple of 10

In [ ]:
# Plotting energies:
# =============================================
# ---- Plot 1: Kinetic Energy ----
plt.figure(figsize=(7, 5))
plt.plot(KE, color='tab:blue')
plt.xlabel('Time Step')
plt.ylabel('Kinetic Energy (Joules)')
plt.title('Kinetic Energy vs Time')
plt.grid(True)
plt.show()

# ---- Plot 2: Potential Energy ----
plt.figure(figsize=(7, 5))
plt.plot(PE, color='tab:orange')
plt.xlabel('Time Step')
plt.ylabel('Potential Energy (Joules)')
plt.title('Potential Energy vs Time')
plt.grid(True)
plt.show()

# ---- Plot 3: Total Energy ----
plt.figure(figsize=(7, 5))
plt.plot(TE, color='tab:green')
plt.xlabel('Time Step')
plt.ylabel('Total Energy (Joules)')
plt.title('Total Energy vs Time')
plt.grid(True)
plt.show()

plt.figure(figsize=(7, 5))
plt.plot(TE, label='Total Energy', color='tab:green')
plt.plot(KE, label='Kinetic Energy', color='tab:blue')
plt.plot(PE, label='Potential Energy', color='tab:orange')
plt.xlabel('Time Step')
plt.ylabel('Energy (reduced units)')
plt.title('Energy Conservation (Velocity-Verlet, LJ Reduced)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Call the radial distribution function

r, g = radial_distribution(traj, L, dr=0.02)
plt.plot(r, g)
plt.xlabel('r (reduced units)')
plt.ylabel('g(r)')
plt.title('Radial distribution function, LJ (PBC)')
plt.show()

In [ ]:
# Write xyz to the file

write_extended_xyz("traj_ext.xyz", traj, L)